In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import json
import pickle
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50,preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.models import Model,load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input,Dense,Embedding,LSTM,Dropout
from keras.layers.merge import add

Using TensorFlow backend.


In [2]:
model = load_model("model_weights/model9.h5")

In [3]:
model_temp = ResNet50(weights="imagenet",input_shape = (224,224,3))

In [4]:
model_resnet = Model(model_temp.input,model_temp.layers[-2].output)

In [5]:
def preprocess_image(img):
    img = image.load_img(img,target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img,axis=0)
    img = preprocess_input(img)
    return img

In [7]:
def encode_image(img):
    img = preprocess_image(img)
    feature_vector = model_resnet.predict(img)
    feature_vector = feature_vector.reshape(1,feature_vector.shape[1])
    return feature_vector

In [9]:
def predict_caption(photo):
    in_text = "startseq"
    max_len = 33
    for i in range(max_len):
        sequence = [word_to_idx[w] for w in in_text.split() if w in word_to_idx]
        sequence = pad_sequences([sequence],maxlen=max_len,padding="post")
        
        y_pred = model.predict([photo,sequence])
        y_pred = y_pred.argmax()
        word = idx_to_word[y_pred]
        in_text+= " "+word
        if word=="endseq":
            break
    final_caption = in_text.split()
    final_caption = final_caption[1:-1]
    final_caption = " ".join(final_caption)
    
    return final_caption

In [10]:
enc = encode_image("Test.jpg")

In [11]:
enc.shape

(1, 2048)

In [13]:
with open("./word_to_idx.pkl","rb") as w2i:
    word_to_idx = pickle.load(w2i)

In [15]:
with open("./idx_to_word.pkl","rb") as i2w:
    idx_to_word = pickle.load(i2w)

In [16]:
predict_caption(enc)

'brown dog is playing with fuzzy on the green grass with its mouth open and tongue sticking on the ground next to the dog and another dog looks at the other of'